In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
def parse(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    return soup

In [3]:
start = "https://www.yogajournal.com/poses/types/"
soup = parse(start)

In [4]:
len()

TypeError: len() takes exactly one argument (0 given)

In [5]:
l_types = soup.find_all(class_="c-block-standard")

In [6]:
type_pose = l_types[0]
json.loads(type_pose.find(class_='o-heading-link')
           ['data-analytics-data'])['props']

{'domain': '<<analytics_vars.domain>>',
 'path': 'https://www.yogajournal.com/poses/types/arm-balances/',
 'listing_type': 'category',
 'location': 'taxonomy_reference',
 'title': 'Arm Balance Yoga Poses'}

In [131]:
def get_types(soup):
    try:
        l_single_pose = soup.find_all(class_="c-article-lead u-breakout")
        if len(l_single_pose) == 1:
            sanskrit_name = soup.find("h3",text = "Sanskrit Name")
            sanskrit_name = sanskrit_name.next_sibling.next_sibling.text if sanskrit_name is not None else ''
            return [(soup.find(class_="c-article-headings u-spacing u-center-block").find("h1").text,
                     l_single_pose[0].find(class_="lazy")["data-iesrc"], 
                     l_single_pose[0].find(class_="lazy")["data-iesrc"],
                     sanskrit_name)]

        l_types = soup.find_all(
            class_="c-block c-block-standard u-align--center c-block--inline-until-medium c-block--inline--mini")
        if len(l_types) == 0:
            l_types = soup.find_all(
                class_="c-block c-block-standard u-align--center c-block--inline-until-medium")
        l_allPose = []
        for type_pose in l_types:
            pose_picture = type_pose.find(
                class_='o-heading-link')
            dic_data = json.loads(pose_picture['data-analytics-data'])['props']
            pose_picture_source = type_pose.find("source")['srcset']
            l_allPose.append((dic_data["title"], 
                              dic_data["path"], 
                              pose_picture_source,
                             ''))
        return l_allPose
    except:
        return None

In [132]:
soup.find_all(class_="c-article-lead u-breakout")

[]

In [133]:
soup = parse(start)
l_allPose = get_types(soup)
all_pose = []
while len(l_allPose) > 0:
    pose_type = l_allPose.pop()
    url = pose_type[1]
    r = requests.get(url)
    if r.url == 'https://www.yogajournal.com/poses/':
        all_pose.append((pose_type[0],pose_type[0].split('|')[-1], pose_type[2],'',url))
        continue
    soup = BeautifulSoup(r.text, "html.parser")
    l_sub_types = get_types(soup)
    if l_sub_types is None:
        continue
    if len(l_sub_types) == 1:
        all_pose.append((pose_type[0], l_sub_types[0][0], l_sub_types[0][1],l_sub_types[0][3],url))
        continue
    else:
        if len(l_sub_types) == 1:
            continue
        for sub_type in l_sub_types:
            l_allPose.append(
                (pose_type[0] + '|' + sub_type[0], sub_type[1], sub_type[2],sub_type[3],url))

In [134]:
len(set([x[1] for x in all_pose]))

115

In [135]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.DataFrame(all_pose)
df.columns = ['classif','pose_name','pose_url','sanskrit name','url2']

In [136]:

pose_classif = {}
for x in all_pose:
    pose_classif[x[1]] = pose_classif.get(x[1], []) + [x[0]]

In [137]:
pose_third = []
for x in all_pose:
    if x[1] != x[0].split('|')[-1]:
        pose_third.append(x)

In [138]:
set([len(x[0].split('|')) for x in all_pose])

{2}

In [139]:
import csv
with open('../data/yogaJournal.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write multiple rows
    writer.writerows(all_pose)

In [140]:
!pwd

/Users/yassinelaghrari/code/ylg619/yogAssist/notebooks


In [141]:
import os
import pandas as pd
from os.path import isfile, dirname, join

my_path = "../raw_data/yoga_dataset_links"

from os import listdir
from os.path import isfile, join
onlyfiles = [f"{f}" for f in listdir(my_path) if isfile(join(my_path, f))]

#df = pd.read_csv(onlyfiles[0], delimiter = "\t", header=None)
#df

In [142]:
dic_files = {}
for x in onlyfiles:
    poses = x.replace('.txt','')
    poses = poses.replace('_', ' ')
    poses = poses.split(' or ')
    for pose in poses:
        dic_files[pose.strip().lower().replace(' pose' ,'')] = x

In [143]:
def find_correspondant_filename(english_name,sanskrit_name):
    english_name = english_name.strip().lower().replace(' pose' ,'')
    sanskrit_name = sanskrit_name.strip().lower()
    if english_name in dic_files:
        return dic_files[english_name]
    
    if sanskrit_name != '' and sanskrit_name in dic_files:
        return dic_files[sanskrit_name]
    for key in dic_files.keys():
        if sanskrit_name != '' and (key in sanskrit_name or sanskrit_name in key):
            return dic_files[key]
        if key in english_name or english_name in key:
            return dic_files[key]
        
    
    return ''

In [144]:
df['file_link'] = df[['pose_name','sanskrit name']].apply(lambda x: find_correspondant_filename(dic_files,*x), axis=1)

In [145]:
df[df['file_link']=='']

,classif,pose_name,pose_url,sanskrit name,url2,file_link
0,Yoga Mudras|Wheel of Dharma Seal,Wheel of Dharma Seal,https://www.yogajournal.com/wp-content/uploads...,Dharmachakra Mudra,https://www.yogajournal.com/poses/dharmachakra...,
1,Yoga Mudras|Upright Seal,Upright Seal,https://www.yogajournal.com/wp-content/uploads...,Linga Mudra,https://www.yogajournal.com/poses/linga-mudra/,
2,Yoga Mudras|Unshakeable Trust Seal,Unshakeable Trust Seal,https://www.yogajournal.com/wp-content/uploads...,Vajrapradama Mudra,https://www.yogajournal.com/poses/vajrapradama...,
3,Yoga Mudras|Truth Seal,Truth Seal,https://www.yogajournal.com/wp-content/uploads...,Tattva Mudra,https://www.yogajournal.com/poses/tattva-mudra/,
4,Yoga Mudras|Salutation Seal,Salutation Seal,https://www.yogajournal.com/wp-content/uploads...,Anjali Mudra,https://www.yogajournal.com/poses/salutation-s...,
5,Yoga Mudras|Meditation Seal,Meditation Seal,https://www.yogajournal.com/wp-content/uploads...,Dhyana Mudra,https://www.yogajournal.com/poses/dhyana-mudra/,
6,Yoga Mudras|Lotus Seal,Lotus Seal,https://www.yogajournal.com/wp-content/uploads...,Padma Mudra,https://www.yogajournal.com/poses/padma-mudra/,
7,Yoga Mudras|Life Force Seal,Life Force Seal,https://www.yogajournal.com/wp-content/uploads...,Prana Mudra,https://www.yogajournal.com/poses/prana-mudra/,
8,Yoga Mudras|Kali Mudra,Kali Mudra,https://www.yogajournal.com/wp-content/uploads...,,https://www.yogajournal.com/poses/kali-mudra/,
9,Yoga Mudras|Ganesha Mudra,Ganesha Mudra,https://www.yogajournal.com/wp-content/uploads...,,https://www.yogajournal.com/poses/ganesha-mudra/,


In [146]:
df.loc[91]['pose_url']

'https://www.yogajournal.com/wp-content/uploads/2007/08/2compass-4.jpg?crop=535:301&width=1070&enable=upscale'

In [147]:
dico_map = {
    'Plank Pose – Step by Step Instructions' : 'Plank Pose',
    'King Pigeon Pose' : 'Pigeon Pose',
    'Cow Pose' : 'Cat Cow Pose',
    "One-Legged King Pigeon Pose II" : 'Rajakapotasana',
    'Half Frog Pose' : 'Frog Pose',
    'Revolved Triangle Pose' : 'Extended Revolved Triangle',
    'Pose Dedicated to the Sage Koundinya I' : 'Eka Pada Koundinyanasana I and II',
    "Bharadvaja's Twist" : "Bharadvaja's_Twist_pose",
    'Revolved Side Angle Pose' : 'utthita padangusthasana',
    'Handstand or Downward Facing Tree Pose': 'Adho Mukha Vrksasana',
    'Extended Hand-to-Big-Toe Pose' : 'Utthita Padangusthasana',
    'Monkey Pose' : 'split pose',
    'Lotus Pose':'Sitting pose 1 (normal)',
    'High Lunge, Crescent Variation' : 'Virabhadrasana I'
    
    
    
}

df['pose_name_rework'] = df['pose_name'].apply(lambda x: dico_map[x] if x in dico_map else x)
df['file_link'] = df[['pose_name_rework','sanskrit name']].apply(lambda x: find_correspondant_filename(dic_files,*x), axis=1)

In [148]:
found_pose = set(df[df['file_link']!='']['file_link'])

existing_pose = set(dic_files.values())

In [149]:
[x for x in existing_pose if x not in found_pose]

['Tortoise_Pose.txt',
 'Supta_Virasana_Vajrasana.txt',
 'Scorpion_pose_or_vrischikasana.txt',
 'Upward_Plank_Pose_or_Purvottanasana_.txt',
 'Wind_Relieving_pose_or_Pawanmuktasana.txt',
 'viparita_virabhadrasana_or_reverse_warrior_pose.txt',
 'Akarna_Dhanurasana.txt',
 'Tree_Pose_or_Vrksasana_.txt',
 'Cockerel_Pose.txt',
 'Yogic_sleep_pose.txt',
 'desktop.ini',
 'Downward-Facing_Dog_pose_or_Adho_Mukha_Svanasana_.txt']

In [153]:
manual_add = [['Scorpion_pose_or_vrischikasana.txt',"https://www.yogajournal.com/wp-content/uploads/2012/10/e1.jpg?crop=535:301&width=1070&enable=upscale"],
['Upward_Plank_Pose_or_Purvottanasana_.txt', "https://www.yogajournal.com/wp-content/uploads/2007/08/upward-plank-pose.jpg?crop=1:1&width=250&enable=upscale"],
['Supta_Virasana_Vajrasana.txt', "https://www.yogajournal.com/wp-content/uploads/2008/06/cleanse_278_4_fnlreclining-hero-pose-supta-virasana.jpg?crop=535:301&width=1070&enable=upscale"],
['Tree_Pose_or_Vrksasana_.txt', "https://www.yogajournal.com/wp-content/uploads/2020/12/ccd03542-1.jpg?crop=535:301&width=1070&enable=upscale"],
 ['Downward-Facing_Dog_pose_or_Adho_Mukha_Svanasana_.txt',"https://www.yogajournal.com/wp-content/uploads/2020/12/2hp_290_1721_bjk.jpg?crop=535:301&width=1070&enable=upscale"],
 ["Cockerel_Pose.txt","https://www.yogajournal.com/wp-content/uploads/2018/06/ty-landrum-9.jpg?crop=535:301&width=1070&enable=upscale"]]

In [155]:
data_yoga_journal =df.loc[df['file_link']!='',['file_link','pose_url']].values.tolist() + manual_add

In [156]:
len(data_yoga_journal)


173

In [160]:
img = data_yoga_journal[0]
os.path.join('../raw_data/yoga_journal_links',img[0])

'../raw_data/yoga_journal_links/Eagle_Pose_or_Garudasana_.txt'

In [164]:
i = 1
for img in data_yoga_journal:
    with open(os.path.join('../raw_data/yoga_journal_links',img[0]),'a') as fy:
        fy.write(f'{img[0].replace(".txt","")}/yogajournal_{i}.jpg\t{img[1]}')
    i+=1
    